## csv 데이터 로딩 복습

In [132]:
# 벤츠 예제 데이터 가공

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# csv 파일 올리기
data = pd.read_csv("./201026_mercedes_traindata.csv") 
# oneshot encoding 할 분류 분리
cf = data.select_dtypes(include=['object']).columns
# 원래 있던 분류 지우고 oneshot encoding 한 결과 붙이기
data_df = pd.concat([data.drop(cf,axis=1),
                     pd.get_dummies(data[cf])],axis=1)
# y 삭제, 분리
X_df, y_df = data_df.drop(['y'],axis=1), data_df['y']
# y 를 0, 1 로 표시
y_df = (y_df<np.median(y_df)).astype(float)
# np 로 형태 변경
X, y = X_df.values, y_df.values

# data 분리
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, 
                                                    random_state=0, # 무작위 분리를 위한 시드 넘버
                                                    stratify=y)  # 1 이 균등하게 분할되도록 data 분리
X_test, X_val, y_test, y_val = train_test_split(X_test,y_test,test_size=0.5,
                                                random_state=0,
                                               stratify=y_test)
print(X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape)
X_train, X_val


(3367, 564) (421, 564) (421, 564) (3367,) (421,) (421,)


(array([[ 801,    0,    0, ...,    0,    0,    0],
        [5072,    0,    0, ...,    0,    0,    0],
        [7320,    0,    0, ...,    0,    0,    0],
        ...,
        [7288,    0,    0, ...,    0,    0,    0],
        [4514,    0,    0, ...,    0,    0,    0],
        [1353,    0,    0, ...,    1,    0,    0]], dtype=int64),
 array([[4407,    0,    0, ...,    0,    0,    0],
        [ 312,    0,    0, ...,    0,    0,    0],
        [7790,    0,    0, ...,    0,    0,    0],
        ...,
        [ 342,    0,    0, ...,    0,    0,    0],
        [7099,    0,    0, ...,    0,    0,    0],
        [ 190,    0,    0, ...,    0,    0,    0]], dtype=int64))

.

## DNN 모델 - hyper parameter 넣어보기
- Early stop (patience, monitor)
- Model check (monitor)
- Learning rate decay 
- Regularization (람다)
- Initializer (he)
- Cross validation (K)

In [120]:
# learning rate decay 함수

def scheduler (epoch, lr):
    if epoch == 20 or epoch == 40 :
        lr = lr * 0.1
    else:
        lr=lr
    return lr

In [176]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold  # 단층 구조로 된 K 개로 접는 거



# hyper parameter 늘어놓기
lambda_list = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
hnode_list = [10, 100, 128, 256]
batsize_list = [32, 128, 256]

# 9개로 나누어서 cross validation
skf = StratifiedKFold(n_splits=9)     
# cross validation 을 위한 데이터 준비 - train + test
X_cval = np.append(X_train, X_val, axis=0)
#print(X_cval.shape, X_train.shape, X_test.shape)
y_cval = np.append(y_train, y_val, axis=0)
#print(y_cval.shape, y_train.shape, y_test.shape)

# 요로케 찍어보면 cross validation 데이터 확인 가능
#np.array(list(skf.split(X_cval,y_cval)))

# 학습 결과 저장소
results = pd.DataFrame()

for hnode in hnode_list:                        # hidden layer 노드 개수 돌리기
    for lambda_ in lambda_list:                 # 람다 값 돌리기
        for batsize in batsize_list:            # batch size 값 돌리기
            acc_list = []
            loss_list = []

            for split_idx, (train_idx, val_idx) in enumerate(skf.split(X_cval,y_cval)):           # cross vali 9개로 돌리기
                X_train, y_train = X_cval[train_idx], y_cval[train_idx]
                X_val, y_val = X_cval[val_idx], y_cval[val_idx]
                #print(X_cval.shape, X_train.shape, X_val.shape, y_cval.shape, y_train.shape, y_test.shape)
        
                model_DNN = Sequential()
                model_DNN.add(Dense(hnode, activation='relu',kernel_initializer='he_normal',       # initializer default 는 Xavier
                                    kernel_regularizer=l2(lambda_),                              # w 에 대한 람다
                                    bias_regularizer=l2(lambda_)))                               # b 에 대한 람다    model_DNN.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(lambda_),bias_regularizer=l2(lambda_)))
                model_DNN.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(lambda_),bias_regularizer=l2(lambda_)))
                model_DNN.compile(optimizer='adam',loss='binary_crossentropy',metrics='acc')
    
                DNN_ES_callback = EarlyStopping(mode='auto', patience=10, monitor='val_loss')
                DNN_MC_callback = ModelCheckpoint('C:/Users/NGV05/Desktop/실습/model_save',monitor='loss',save_best_only=True)
                DNN_LRS_callback = LearningRateScheduler(schedule=scheduler)
    
                history = model_DNN.fit(X,y,verbose=0,validation_data=(X_val,y_val), epochs=10,
                                        batch_size=batsize, 
                                        callbacks=[DNN_ES_callback,DNN_MC_callback,DNN_LRS_callback])
    
                loss, acc = model_DNN.evaluate(X_val,y_val)
                acc_list.append(acc)
                loss_list.append(loss)
    
            avg_acc = np.mean(acc_list)
            avg_loss = np.mean(loss_list)
            result = pd.DataFrame([[hnode,lambda_,batsize,avg_acc,avg_loss]],columns=('hnode','lambda_','batsize','acc','loss'))
            results = results.append(result)

results.to_csv('C:/Users/NGV05/Desktop/실습/201027_results.csv')

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 644us/step - loss: 0.4622 - acc: 0.7831
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 664us/step - loss: 0.3537 - acc: 0.8859
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV

40/40 [==============================] - 0s 649us/step - loss: 0.4903 - acc: 0.8179
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 649us/step - loss: 0.4414 - acc: 0.8773
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/User

40/40 [==============================] - 0s 822us/step - loss: 0.6927 - acc: 0.7805
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 888us/step - loss: 0.8718 - acc: 0.8211
INFO:tensorflow:Assets written to: C:/User

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 1ms/step - loss: 1.8918 - acc: 0.5637
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05

...

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 671us/step - loss: 0.6625 - acc: 0.7395
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 672us/step - loss: 1.9719 - acc: 0.5666
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 799us/step - loss: 1.9436 - acc: 0.5534
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/User

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 666us/step - loss: 2.4482 - acc: 0.5309
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 1ms/step - loss: 31.3064 - acc: 0.5012
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV0

INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
40/40 [==============================] - 0s 890us/step - loss: 218.6537 - acc: 0.4956
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/NGV05/Desktop/실습\model_save\assets
INFO:tensorflow:Assets written to: C:/Users/N

In [177]:
results

,hnode,lambda_,batsize,acc,loss
0,10,0.0001,32,0.852174,0.367974
0,10,0.0001,128,0.835263,0.435370
0,10,0.0001,256,0.815467,0.479008
0,10,0.0010,32,0.877774,0.369903
0,10,0.0010,128,0.817841,0.454035
...,...,...,...,...,...
0,256,10.0000,128,0.501056,12.574457
0,256,10.0000,256,0.500264,31.435558
0,256,100.0000,32,0.498945,1.331825
0,256,100.0000,128,0.499736,31.482255


.

## 저장한 모델 읽어오기

In [179]:
from tensorflow.keras.models import load_model

model_loaded = load_model('C:/Users/NGV05/Desktop/실습/model_save')
model_loaded.evaluate(X_test,y_test)

14/14 [==============================] - 0s 2ms/step - loss: 237.4626 - acc: 0.5012


[237.4625701904297, 0.5011876225471497]

.

## 연습용

In [168]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.regularizers import l2

lambda_ = 0.01
h = 128
model_DNN = Sequential()

model_DNN.add(Dense(h, activation='relu',kernel_initializer='he_normal',       # initializer default 는 Xavier
                   kernel_regularizer=l2(lambda_),                              # w 에 대한 람다
                   bias_regularizer=l2(lambda_)))                               # b 에 대한 람다
model_DNN.add(Dense(1,activation='sigmoid',
                    kernel_regularizer=l2(lambda_),
                    bias_regularizer=l2(lambda_)))
model_DNN.compile(optimizer='adam',loss='binary_crossentropy',metrics='acc')

# early stop 조건
DNN_ES_callback = EarlyStopping(mode='auto', patience=10, monitor='val_loss')
# 가장 좋은 모델 저장 조건
DNN_MC_callback = ModelCheckpoint('C:/Users/NGV05/Desktop/실습/model_save',monitor='loss',save_best_only=True)
# learning rate decay 조절 조건
DNN_LRS_callback = LearningRateScheduler(schedule=scheduler)

history = model_DNN.fit(X,y,epochs=50, validation_data=(X_val,y_val), callbacks=[DNN_MC_callback,DNN_ES_callback,DNN_LRS_callback])

Epoch 1/50
132/132 [==============================] - 0s 2ms/step - loss: 15.6530 - acc: 0.6070 - val_loss: 1.5019 - val_acc: 0.6033
Epoch 2/50
132/132 [==============================] - 0s 1ms/step - loss: 1.2833 - acc: 0.7170 - val_loss: 0.6825 - val_acc: 0.8765
Epoch 3/50
132/132 [==============================] - 0s 1ms/step - loss: 1.2902 - acc: 0.7220 - val_loss: 0.7114 - val_acc: 0.8575
Epoch 4/50
132/132 [==============================] - 0s 984us/step - loss: 0.6785 - acc: 0.8161 - val_loss: 1.4911 - val_acc: 0.6057
Epoch 5/50
132/132 [==============================] - 0s 1ms/step - loss: 1.8145 - acc: 0.7080 - val_loss: 0.9865 - val_acc: 0.7078
Epoch 6/50
132/132 [==============================] - 0s 1ms/step - loss: 0.8018 - acc: 0.7992 - val_loss: 1.3490 - val_acc: 0.6176
Epoch 7/50
132/132 [==============================] - 0s 1ms/step - loss: 0.8933 - acc: 0.7864 - val_loss: 0.4939 - val_acc: 0.8812
Epoch 8/50
132/132 [==============================] - 0s 902us/step - los

.

## KNeighborsClassifier

In [234]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

XI, yI = load_iris(return_X_y=True)
XI_train, XI_test, yI_train, yI_test = train_test_split(XI,yI,test_size=0.2,random_state=0,stratify=yI)

print(XI_train.shape, XI_test.shape)
print(yI_train.shape, yI_test)

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(XI_train, yI_train)


for i in XI_test:
    print(neigh.predict_proba(i.reshape(1,4)))
print(neigh.score(XI_test,yI_test))


(120, 4) (30, 4)
(120,) [0 1 0 2 0 1 2 0 0 1 2 1 1 2 1 2 2 1 1 0 0 2 2 2 0 1 1 2 0 0]
[[1. 0. 0.]]
[[0. 1. 0.]]
[[1. 0. 0.]]
[[0.         0.33333333 0.66666667]]
[[1. 0. 0.]]
[[0. 1. 0.]]
[[0. 0. 1.]]
[[1. 0. 0.]]
[[1. 0. 0.]]
[[0. 1. 0.]]
[[0. 0. 1.]]
[[0. 1. 0.]]
[[0. 1. 0.]]
[[0. 0. 1.]]
[[0. 1. 0.]]
[[0.         0.33333333 0.66666667]]
[[0.         0.33333333 0.66666667]]
[[0. 1. 0.]]
[[0. 1. 0.]]
[[1. 0. 0.]]
[[1. 0. 0.]]
[[0. 0. 1.]]
[[0. 0. 1.]]
[[0. 0. 1.]]
[[1. 0. 0.]]
[[0. 1. 0.]]
[[0. 1. 0.]]
[[0. 0. 1.]]
[[1. 0. 0.]]
[[1. 0. 0.]]
1.0
